## Einbettungen

In unserem vorherigen Beispiel haben wir mit hochdimensionalen Bag-of-Words-Vektoren der Länge `vocab_size` gearbeitet und explizit von niedrigdimensionalen Positionsdarstellungsvektoren in spärliche One-Hot-Darstellungen umgewandelt. Diese One-Hot-Darstellung ist nicht speichereffizient. Außerdem wird jedes Wort unabhängig von den anderen behandelt, d.h. One-Hot-codierte Vektoren drücken keine semantische Ähnlichkeit zwischen Wörtern aus.

In dieser Einheit werden wir weiterhin den **News AG**-Datensatz untersuchen. Zu Beginn laden wir die Daten und holen einige Definitionen aus dem vorherigen Notebook.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Was ist ein Embedding?

Die Idee des **Embeddings** besteht darin, Wörter durch niedrigdimensionale, dichte Vektoren darzustellen, die in gewisser Weise die semantische Bedeutung eines Wortes widerspiegeln. Später werden wir besprechen, wie man sinnvolle Wort-Embeddings erstellt, aber vorerst betrachten wir Embeddings einfach als eine Methode, die Dimensionalität eines Wortvektors zu reduzieren.

Eine Embedding-Schicht nimmt also ein Wort als Eingabe und erzeugt einen Ausgabevektor mit der angegebenen `embedding_size`. In gewisser Weise ähnelt sie einer `Linear`-Schicht, aber anstatt einen One-Hot-codierten Vektor zu verwenden, kann sie eine Wortnummer als Eingabe akzeptieren.

Indem wir die Embedding-Schicht als erste Schicht in unserem Netzwerk verwenden, können wir vom Bag-of-Words-Modell zum **Embedding-Bag-Modell** wechseln. Dabei wird jedes Wort in unserem Text zunächst in das entsprechende Embedding umgewandelt, und anschließend wird eine Aggregationsfunktion wie `sum`, `average` oder `max` über alle diese Embeddings berechnet.

![Bild, das einen Embedding-Klassifikator für fünf Sequenzwörter zeigt.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Unser neuronales Klassifikationsnetzwerk beginnt mit einer Embedding-Schicht, gefolgt von einer Aggregationsschicht und einem linearen Klassifikator darüber:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Umgang mit variabler Sequenzgröße

Aufgrund dieser Architektur müssen Minibatches für unser Netzwerk auf eine bestimmte Weise erstellt werden. In der vorherigen Einheit, bei der Verwendung von Bag-of-Words, hatten alle BoW-Tensoren in einem Minibatch die gleiche Größe `vocab_size`, unabhängig von der tatsächlichen Länge unserer Textsequenz. Sobald wir zu Wort-Embeddings wechseln, haben wir eine variable Anzahl von Wörtern in jeder Textprobe, und beim Kombinieren dieser Proben in Minibatches müssen wir eine Auffüllung (Padding) anwenden.

Dies kann durch die gleiche Technik erreicht werden, indem eine `collate_fn`-Funktion an die Datenquelle übergeben wird:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Training des Einbettungs-Klassifikators

Nun, da wir einen geeigneten Dataloader definiert haben, können wir das Modell mit der Trainingsfunktion trainieren, die wir in der vorherigen Einheit definiert haben:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Hinweis**: Wir trainieren hier nur mit 25.000 Datensätzen (weniger als eine vollständige Epoche) aus Zeitgründen, aber Sie können das Training fortsetzen, eine Funktion schreiben, um über mehrere Epochen zu trainieren, und mit dem Lernratenparameter experimentieren, um eine höhere Genauigkeit zu erreichen. Sie sollten in der Lage sein, eine Genauigkeit von etwa 90 % zu erreichen.


### EmbeddingBag-Schicht und Darstellung von Sequenzen variabler Länge

In der vorherigen Architektur mussten wir alle Sequenzen auf die gleiche Länge auffüllen, um sie in ein Minibatch einzupassen. Dies ist jedoch nicht die effizienteste Methode, um Sequenzen variabler Länge darzustellen – ein alternativer Ansatz wäre die Verwendung eines **Offset-Vektors**, der die Offsets aller Sequenzen in einem großen Vektor speichert.

![Bild, das eine Offset-Sequenzdarstellung zeigt](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Hinweis**: Auf dem obigen Bild zeigen wir eine Zeichenfolge, aber in unserem Beispiel arbeiten wir mit Wortsequenzen. Das allgemeine Prinzip, Sequenzen mit einem Offset-Vektor darzustellen, bleibt jedoch dasselbe.

Um mit der Offset-Darstellung zu arbeiten, verwenden wir die [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html)-Schicht. Sie ähnelt der `Embedding`-Schicht, nimmt jedoch einen Inhaltsvektor und einen Offset-Vektor als Eingabe und enthält außerdem eine Aggregationsschicht, die `mean`, `sum` oder `max` sein kann.

Hier ist ein modifiziertes Netzwerk, das `EmbeddingBag` verwendet:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Um den Datensatz für das Training vorzubereiten, müssen wir eine Umrechnungsfunktion bereitstellen, die den Offset-Vektor vorbereitet:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Beachten Sie, dass unser Netzwerk im Gegensatz zu allen vorherigen Beispielen jetzt zwei Parameter akzeptiert: Datenvektor und Offsetvektor, die unterschiedliche Größen haben. Ebenso liefert uns unser Datenlader jetzt 3 Werte anstelle von 2: Sowohl Text- als auch Offsetvektoren werden als Features bereitgestellt. Daher müssen wir unsere Trainingsfunktion geringfügig anpassen, um dies zu berücksichtigen:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantische Einbettungen: Word2Vec

In unserem vorherigen Beispiel hat die Einbettungsschicht des Modells gelernt, Wörter in Vektorrepräsentationen umzuwandeln. Diese Repräsentationen hatten jedoch nicht viel semantische Bedeutung. Es wäre wünschenswert, solche Vektorrepräsentationen zu lernen, bei denen ähnliche Wörter oder Synonyme Vektoren entsprechen, die in Bezug auf eine bestimmte Vektordistanz (z. B. euklidische Distanz) nahe beieinander liegen.

Um dies zu erreichen, müssen wir unser Einbettungsmodell auf eine große Textsammlung in einer spezifischen Weise vortrainieren. Eine der ersten Methoden, um semantische Einbettungen zu trainieren, wird [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) genannt. Sie basiert auf zwei Hauptarchitekturen, die verwendet werden, um eine verteilte Repräsentation von Wörtern zu erzeugen:

- **Continuous Bag-of-Words** (CBoW) — In dieser Architektur trainieren wir das Modell darauf, ein Wort aus dem umgebenden Kontext vorherzusagen. Gegeben das N-Gramm $(W_{-2},W_{-1},W_0,W_1,W_2)$, ist das Ziel des Modells, $W_0$ aus $(W_{-2},W_{-1},W_1,W_2)$ vorherzusagen.
- **Continuous Skip-Gram** ist das Gegenteil von CBoW. Das Modell verwendet das umgebende Fenster von Kontextwörtern, um das aktuelle Wort vorherzusagen.

CBoW ist schneller, während Skip-Gram langsamer ist, aber eine bessere Repräsentation für seltene Wörter liefert.

![Bild, das sowohl die CBoW- als auch die Skip-Gram-Algorithmen zur Umwandlung von Wörtern in Vektoren zeigt.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Um mit Word2Vec-Einbettungen zu experimentieren, die auf dem Google-News-Datensatz vortrainiert wurden, können wir die **gensim**-Bibliothek verwenden. Unten finden wir die Wörter, die 'neural' am ähnlichsten sind.

> **Hinweis:** Wenn Sie zum ersten Mal Wortvektoren erstellen, kann das Herunterladen einige Zeit in Anspruch nehmen!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Wir können auch Vektoreinbettungen aus dem Wort berechnen, die zur Schulung des Klassifikationsmodells verwendet werden (wir zeigen nur die ersten 20 Komponenten des Vektors zur besseren Übersicht):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Das Großartige an semantischen Einbettungen ist, dass man die Vektorkodierung manipulieren kann, um die Semantik zu ändern. Zum Beispiel können wir nach einem Wort suchen, dessen Vektorrepräsentation so nah wie möglich an den Wörtern *König* und *Frau* liegt und so weit wie möglich vom Wort *Mann* entfernt ist:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Sowohl CBoW als auch Skip-Grams sind „prädiktive“ Einbettungen, da sie nur lokale Kontexte berücksichtigen. Word2Vec nutzt den globalen Kontext nicht aus.

**FastText** baut auf Word2Vec auf, indem es Vektorrepräsentationen für jedes Wort und die Zeichen-n-Gramme innerhalb jedes Wortes lernt. Die Werte der Repräsentationen werden dann bei jedem Trainingsschritt zu einem Vektor gemittelt. Obwohl dies eine Menge zusätzlicher Berechnungen während des Pre-Trainings erfordert, ermöglicht es den Wort-Einbettungen, Subwort-Informationen zu kodieren.

Eine andere Methode, **GloVe**, nutzt die Idee der Ko-Vorkommensmatrix und verwendet neuronale Methoden, um die Ko-Vorkommensmatrix in ausdrucksstärkere und nicht-lineare Wortvektoren zu zerlegen.

Du kannst mit dem Beispiel experimentieren, indem du die Einbettungen auf FastText und GloVe änderst, da gensim mehrere verschiedene Modelle für Wort-Einbettungen unterstützt.


## Verwendung vortrainierter Einbettungen in PyTorch

Wir können das obige Beispiel so anpassen, dass die Matrix in unserer Einbettungsschicht mit semantischen Einbettungen wie Word2Vec vorab gefüllt wird. Dabei müssen wir berücksichtigen, dass die Vokabulare der vortrainierten Einbettungen und unseres Textkorpus wahrscheinlich nicht übereinstimmen. Daher werden wir die Gewichte für die fehlenden Wörter mit Zufallswerten initialisieren:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Jetzt lassen Sie uns unser Modell trainieren. Beachten Sie, dass die Zeit, die zum Trainieren des Modells benötigt wird, aufgrund der größeren Größe der Einbettungsschicht und damit der deutlich höheren Anzahl von Parametern erheblich länger ist als im vorherigen Beispiel. Außerdem müssen wir möglicherweise unser Modell mit mehr Beispielen trainieren, wenn wir Überanpassung vermeiden wollen.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

In unserem Fall sehen wir keinen großen Anstieg der Genauigkeit, was wahrscheinlich auf sehr unterschiedliche Vokabulare zurückzuführen ist.  
Um das Problem der unterschiedlichen Vokabulare zu lösen, können wir eine der folgenden Lösungen verwenden:  
* Das Word2Vec-Modell mit unserem Vokabular neu trainieren  
* Unser Dataset mit dem Vokabular des vortrainierten Word2Vec-Modells laden. Das Vokabular, das zum Laden des Datasets verwendet wird, kann während des Ladens angegeben werden.  

Der letztere Ansatz scheint einfacher zu sein, insbesondere weil das PyTorch-Framework `torchtext` integrierte Unterstützung für Embeddings bietet. Wir können beispielsweise ein GloVe-basiertes Vokabular auf folgende Weise instanziieren:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Das geladene Vokabular bietet die folgenden grundlegenden Operationen:
* Das `vocab.stoi`-Wörterbuch ermöglicht es uns, ein Wort in seinen Index im Wörterbuch umzuwandeln.
* `vocab.itos` macht das Gegenteil – es wandelt eine Zahl in ein Wort um.
* `vocab.vectors` ist das Array der Einbettungsvektoren. Um die Einbettung eines Wortes `s` zu erhalten, müssen wir `vocab.vectors[vocab.stoi[s]]` verwenden.

Hier ist ein Beispiel für die Manipulation von Einbettungen, um die Gleichung **kind-man+woman = queen** zu demonstrieren (ich musste den Koeffizienten ein wenig anpassen, damit es funktioniert):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Um den Klassifikator mit diesen Einbettungen zu trainieren, müssen wir zunächst unseren Datensatz mit dem GloVe-Vokabular codieren:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Wie wir oben gesehen haben, werden alle Vektoreinbettungen in der `vocab.vectors`-Matrix gespeichert. Dadurch wird es sehr einfach, diese Gewichte durch einfaches Kopieren in die Gewichte der Einbettungsschicht zu laden:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Lassen Sie uns nun unser Modell trainieren und sehen, ob wir bessere Ergebnisse erzielen:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Einer der Gründe, warum wir keine signifikante Steigerung der Genauigkeit sehen, liegt darin, dass einige Wörter aus unserem Datensatz im vortrainierten GloVe-Vokabular fehlen und daher im Wesentlichen ignoriert werden. Um dies zu überwinden, können wir eigene Embeddings auf unserem Datensatz trainieren.


## Kontextuelle Einbettungen

Eine zentrale Einschränkung traditioneller vortrainierter Einbettungsrepräsentationen wie Word2Vec ist das Problem der Bedeutungsunterscheidung von Wörtern. Während vortrainierte Einbettungen einen Teil der Bedeutung von Wörtern im Kontext erfassen können, wird jede mögliche Bedeutung eines Wortes in derselben Einbettung kodiert. Dies kann in nachgelagerten Modellen zu Problemen führen, da viele Wörter, wie das Wort „play“, je nach Kontext unterschiedliche Bedeutungen haben.

Zum Beispiel hat das Wort „play“ in den folgenden zwei Sätzen eine ganz unterschiedliche Bedeutung:
- Ich war in einem **Theaterstück**.
- John möchte mit seinen Freunden **spielen**.

Die oben genannten vortrainierten Einbettungen repräsentieren beide Bedeutungen des Wortes „play“ in derselben Einbettung. Um diese Einschränkung zu überwinden, müssen wir Einbettungen basierend auf dem **Sprachmodell** erstellen, das auf einem großen Textkorpus trainiert wurde und *versteht*, wie Wörter in unterschiedlichen Kontexten zusammengefügt werden können. Die Diskussion über kontextuelle Einbettungen liegt außerhalb des Umfangs dieses Tutorials, aber wir werden darauf zurückkommen, wenn wir in der nächsten Einheit über Sprachmodelle sprechen.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
